# PCA Notebook
Note we're using song as the bag for these tables

## Setup

In [1]:
import numpy as np
import pandas as pd
import plotly_express as px
from sklearn.decomposition import PCA
from scipy.linalg import norm, eigh

OHCO = ['album_id', 'song_num', 'stanza_num', 'line_num', 'token_num']
colors = 'YlGnBu'

## Functions

In [2]:
def get_PCA(X:pd.DataFrame, k:int, norm_docs=True, center_by_mean=False, center_by_variance=False) -> tuple:
    '''
    Function to compute PCA on a given document-term count matrix.

    PARAMETERS:

    X - pandas `DataFrame` representing document-term count matrix to perform PCA on

    k - int number of principal components to return

    norm_docs - bool (defaults to True) of whether to normalize document length. Uses L2 norm

    center_by_mean - bool (defaults to False) of whether to center term vectors by column mean. Uses pandas `.cov()` method

    center_by_variance - bool (defaults to False) of whether to center term vectors by column variance. Uses pandas `.cov()` method

    
    OUTPUTS:

    LOADINGS - pandas `DataFrame` representing term-component matrix

    DCM - pandas `DataFrame` representing document-component matrix

    COMPINF - pandas `DataFrame` representing component information
    '''

    # Input handling to ensure only one centering method is used
    if (center_by_mean and center_by_variance):
        raise SyntaxError('Cannot center by both mean and variance')
    
    # Normalize docs
    if (norm_docs):
        tfidf = (X.T/norm(X, 2, axis=1)).T
    else:
        tfidf = X
    
    # Center by mean
    if (center_by_mean):
        tfidf = tfidf - tfidf.mean()

    # Center by variance
    if (center_by_variance):
        tfidf = tfidf - tfidf.var()
    
    # Compute variance-covariange matrix
    COV = tfidf.cov()

    # Eigendecomposition
    eig_vals, eig_vecs = eigh(COV)

    # Convernt eigenvalues and eigenvectors into DataFrames
    EIG_VECS = pd.DataFrame(eig_vecs, index=COV.index, columns=COV.index)
    EIG_VALS = pd.DataFrame(eig_vals, index=COV.index, columns=['eig_val'])
    EIG_PAIRS = EIG_VALS.join(EIG_VECS.T)\
        .sort_values('eig_val', ascending=False)
    
    EIG_PAIRS['exp_var'] = np.round((EIG_PAIRS.eig_val / EIG_PAIRS.eig_val.sum()) * 100, 2)

    # Select top k components
    COMPS = EIG_PAIRS.sort_values('exp_var', ascending=False).head(k).reset_index(drop=True)
    COMPS.index.name = 'comp_id'
    COMPS.index = ["PC{}".format(i) for i in COMPS.index.tolist()]
    COMPS.index.name = 'pc_id'

    # Create LOADINGS
    LOADINGS = COMPS[COV.index].T
    LOADINGS.index.name = 'term_str'

    # Create DCM
    DCM = tfidf.dot(COMPS[COV.index].T)

    # Create COMPINF
    top_terms = []
    for i in range(k):
        for j in [0, 1]:
            comp_str = ' '.join(LOADINGS.sort_values(f'PC{i}', ascending=bool(j)).head(10).index.to_list())
            top_terms.append((f"PC{i}", j, comp_str))
    COMPINF = pd.DataFrame(top_terms).set_index([0,1]).unstack()
    COMPINF.index.name = 'comp_id'
    COMPINF.columns = COMPINF.columns.droplevel(0) 
    COMPINF = COMPINF.rename(columns={0:'pos', 1:'neg'})

    return (LOADINGS, DCM, COMPINF)

In [3]:
def vis_pcs(M, a, b, label='artist', hover_name='title', symbol=None, size=None):
    return px.scatter(M, f"PC{a}", f"PC{b}", color=label, hover_data=['artist', 'title', 'album', 'track_number'],
                     symbol=symbol, size=size,
                     marginal_x='box', height=800)

In [4]:
def vis_loadings(a=0, b=1, hover_name='term_str'):
    X = LOADINGS.join(VOCAB)
    #X = LOADINGS.join(VSHORT)
    return px.scatter(X.reset_index(), f"PC{a}", f"PC{b}", 
                      text='term_str', size='i', color='max_pos', 
                      marginal_x='box', height=800)

## Read Data

In [5]:
LIB = pd.read_csv('../tables/LIB.csv', sep='|').set_index('album_id')
SONG_LIB = pd.read_csv('../tables/SONG_LIB', sep='|').set_index(['album_id', 'song_num'])
CORPUS = pd.read_csv('../tables/CORPUS.csv', sep='|').set_index(OHCO)
VOCAB = pd.read_csv('../tables/VOCAB.csv', sep='|').set_index('term_str')
BOW = pd.read_csv('../tables/BOW_SONG.csv', sep='|').set_index(['album_id', 'song_num', 'term_str'])
DTCM = pd.read_csv('../tables/DTCM_SONG.csv', sep='|').set_index(OHCO[:2])
TFIDF = pd.read_csv('../tables/TFIDF_SONG.csv', sep='|').set_index(OHCO[:2])
TFIDF_L2 = pd.read_csv('../tables/TFIDF_L2_SONG.csv', sep='|').set_index(['album_id', 'song_num'])

### Set column indices where relevant

In [6]:
DTCM.columns.name = 'term_str'
TFIDF.columns.name = 'term_str'
TFIDF_L2.columns.name = 'term_str'

In [7]:
TFIDF_L2

term_str              admit    afraid     again  ahead      aint  alien  \
album_id song_num                                                         
0        0         0.080607  0.116604  0.000000    0.0  0.000000    0.0   
         1         0.000000  0.000000  0.549158    0.0  0.000000    0.0   
1        0         0.000000  0.000000  0.000000    0.0  0.000000    0.0   
         1         0.000000  0.000000  0.000000    0.0  0.000000    0.0   
         2         0.000000  0.000000  0.000000    0.0  0.000000    0.0   
...                     ...       ...       ...    ...       ...    ...   
91       5         0.000000  0.000000  0.034126    0.0  0.000000    0.0   
         6         0.000000  0.000000  0.000000    0.0  0.000000    0.0   
         7         0.000000  0.000000  0.000000    0.0  0.000000    0.0   
         8         0.000000  0.000000  0.000000    0.0  0.143172    0.0   
         9         0.000000  0.000000  0.000000    0.0  0.000000    0.0   

term_str           alive  almost  alone   already  ...  year     years  \
album_id song_num                                  ...                   
0        0           0.0     0.0    0.0  0.059938  ...   0.0  0.057292   
         1           0.0     0.0    0.0  0.000000  ...   0.0  0.000000   
1        0           0.0     0.0    0.0  0.000000  ...   0.0  0.000000   
         1           0.0     0.0    0.0  0.000000  ...   0.0  0.000000   
         2           0.0     0.0    0.0  0.000000  ...   0.0  0.000000   
...                  ...     ...    ...       ...  ...   ...       ...   
91       5           0.0     0.0    0.0  0.000000  ...   0.0  0.000000   
         6           0.0     0.0    0.0  0.000000  ...   0.0  0.052607   
         7           0.0     0.0    0.0  0.105323  ...   0.0  0.000000   
         8           0.0     0.0    0.0  0.000000  ...   0.0  0.000000   
         9           0.0     0.0    0.0  0.000000  ...   0.0  0.000000   

term_str           yellow      youd     youll  young     youre  yours  \
album_id song_num                                                       
0        0            0.0  0.000000  0.000000    0.0  0.087363    0.0   
         1            0.0  0.000000  0.000000    0.0  0.000000    0.0   
1        0            0.0  0.000000  0.000000    0.0  0.000000    0.0   
         1            0.0  0.000000  0.000000    0.0  0.000000    0.0   
         2            0.0  0.000000  0.000000    0.0  0.000000    0.0   
...                   ...       ...       ...    ...       ...    ...   
91       5            0.0  0.000000  0.038554    0.0  0.172312    0.0   
         6            0.0  0.000000  0.000000    0.0  0.060164    0.0   
         7            0.0  0.000000  0.000000    0.0  0.076757    0.0   
         8            0.0  0.068694  0.000000    0.0  0.049581    0.0   
         9            0.0  0.000000  0.000000    0.0  0.000000    0.0   

term_str              youth     youve  
album_id song_num                      
0        0         0.000000  0.000000  
         1         0.000000  0.000000  
1        0         0.000000  0.000000  
         1         0.000000  0.000000  
         2         0.000000  0.395009  
...                     ...       ...  
91       5         0.069075  0.038753  
         6         0.000000  0.040593  
         7         0.000000  0.000000  
         8         0.000000  0.100358  
         9         0.000000  0.000000  

[985 rows x 1000 columns]

### Remove NULLs from `TFIDF_L2`

In [8]:
a = len(TFIDF_L2)
TFIDF_L2 = TFIDF_L2.dropna()
b = len(TFIDF_L2)
bag_loss = a - b
bag_loss

8

## Generate `LOADINGS`, `DCM`, and `COMPINF`

In [9]:
LOADINGS, DCM, COMPINF = get_PCA(TFIDF_L2, k=10, norm_docs=False, center_by_mean=False, center_by_variance=False)

In [10]:
LOADINGS.head(10).style.background_gradient(cmap=colors)

pc_id,PC0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9
term_str,,,,,,,,,,
admit,0.002451,-0.000686,-0.002238,0.004360,0.000151,0.002594,0.000301,0.004247,-0.003283,-0.000771
afraid,0.007607,-0.007014,-0.001308,0.041637,-0.043310,0.027658,0.013895,0.020932,-0.015388,0.004629
again,-0.028124,-0.032547,-0.030773,-0.022023,-0.017743,-0.124288,-0.046008,-0.081565,0.018272,-0.037316
ahead,0.005677,0.013225,-0.004436,-0.005088,0.007706,-0.000516,0.005099,-0.004896,0.019412,0.000356
aint,0.002635,0.001359,-0.006889,0.015119,0.003780,0.014053,-0.034688,-0.039753,-0.054490,-0.000657
alien,-0.006210,-0.014133,0.000295,-0.004746,0.008999,-0.006129,-0.027080,-0.001546,0.007760,-0.002238
alive,-0.008496,0.031024,0.021922,-0.002677,-0.005897,0.021213,-0.007178,-0.032587,-0.093107,0.021352
almost,0.003419,-0.007330,-0.020146,0.002769,-0.003448,0.013014,-0.009655,0.045796,0.039020,-0.004275
alone,-0.040164,-0.029219,0.021885,0.066907,0.003694,-0.015398,0.040454,-0.011203,-0.008539,0.054039


In [11]:
DCM.head()

pc_id                   PC0       PC1       PC2       PC3       PC4       PC5  \
album_id song_num                                                               
0        0        -0.156579 -0.108964 -0.148501  0.061985 -0.110232  0.015552   
         1        -0.149733 -0.057213 -0.032084  0.028575 -0.050405 -0.119869   
1        0        -0.009232 -0.032631  0.021213  0.064602  0.025031 -0.014246   
         1        -0.017147  0.001126 -0.020606  0.064183 -0.057723 -0.014021   
         2        -0.043525  0.004738 -0.054372  0.035539 -0.006521  0.021488   

pc_id                   PC6       PC7       PC8       PC9  
album_id song_num                                          
0        0        -0.018331 -0.026064  0.049509  0.051488  
         1        -0.133881 -0.209176 -0.144738 -0.148798  
1        0         0.036429  0.028644  0.025909  0.092896  
         1         0.031238 -0.055770  0.049639  0.000454  
         2         0.036492 -0.051879  0.003264 -0.051876

In [12]:
DCM.tail()

pc_id                   PC0       PC1       PC2       PC3       PC4       PC5  \
album_id song_num                                                               
91       5        -0.116757 -0.171783 -0.136821  0.071665 -0.113748 -0.014061   
         6        -0.068450 -0.148711  0.059569 -0.026940 -0.222486  0.015800   
         7        -0.126655 -0.145567 -0.107183  0.077105 -0.054665  0.006685   
         8        -0.064518 -0.094885 -0.011958  0.045270 -0.092617  0.016906   
         9         0.013646 -0.020556 -0.021711  0.023240 -0.005267  0.017130   

pc_id                   PC6       PC7       PC8       PC9  
album_id song_num                                          
91       5        -0.110583  0.060438  0.027360  0.010152  
         6         0.056047 -0.001361 -0.029336  0.012611  
         7         0.024628  0.009780  0.039644  0.059879  
         8        -0.003498 -0.002390 -0.038139 -0.038723  
         9         0.038854 -0.033734 -0.001290 -0.034217

In [13]:
COMPINF.head()

1,pos,neg
comp_id,,
PC0,light blue black blood leaves girls round noth...,want wanna dont love really youre need know ma...
PC1,want really stars moon sing free room girls ev...,love wanna youre know dont gonna live just wer...
PC2,yeah wanna live feel gonna dead everybody figh...,youre never time away know need come tell noth...
PC3,know live never away take cant need yeah wanna...,love want release stop come mother gonna wont ...
PC4,wanna dont live leave more wish little right f...,yeah gonna were youre know love come time away...


## Project DOC (`SONG_LIB`) onto components

In [14]:
DCM = DCM.join(SONG_LIB)
DCM.head()

PC0       PC1       PC2       PC3       PC4       PC5  \
album_id song_num                                                               
0        0        -0.156579 -0.108964 -0.148501  0.061985 -0.110232  0.015552   
         1        -0.149733 -0.057213 -0.032084  0.028575 -0.050405 -0.119869   
1        0        -0.009232 -0.032631  0.021213  0.064602  0.025031 -0.014246   
         1        -0.017147  0.001126 -0.020606  0.064183 -0.057723 -0.014021   
         2        -0.043525  0.004738 -0.054372  0.035539 -0.006521  0.021488   

                        PC6       PC7       PC8       PC9  ... loudness mode  \
album_id song_num                                          ...                 
0        0        -0.018331 -0.026064  0.049509  0.051488  ...   -6.554    1   
         1        -0.133881 -0.209176 -0.144738 -0.148798  ...   -5.066    0   
1        0         0.036429  0.028644  0.025909  0.092896  ...   -6.520    1   
         1         0.031238 -0.055770  0.049639  0.000454  ...  -13.207    0   
         2         0.036492 -0.051879  0.003264 -0.051876  ...  -10.827    0   

                  speechiness acousticness instrumentalness liveness valence  \
album_id song_num                                                              
0        0             0.0471        0.138            0.657    0.100   0.186   
         1             0.1010        0.216            0.688    0.110   0.726   
1        0             0.0297        0.303            0.272    0.109   0.620   
         1             0.0336        0.968            0.853    0.126   0.113   
         2             0.0269        0.666            0.837    0.117   0.271   

                     tempo  time_signature         album_title  
album_id song_num                                               
0        0         113.018               4               45:33  
         1          94.999               4               45:33  
1        0         148.936               4  A Moon Shaped Pool  
         1         137.561               3  A Moon Shaped Pool  
         2         139.149               4  A Moon Shaped Pool  

[5 rows x 34 columns]

In [15]:
vis_pcs(DCM, 1, 0, label='artist')

In [16]:
vis_loadings(0, 1)